In [22]:
import pandas as pd
import requests
import numpy as np
import requests
from bs4 import BeautifulSoup
import re



In [23]:
sal_data = pd.read_csv('ClickStream+PurchasesCombined_FINAL_15Feb (1).csv')
sal_data["index"]=sal_data.index
#sal_data.columns = ['index', 'url']

c:\VS_Code\O360\O360\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (21,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
sal_data = sal_data[['Path Analysis','index','UUID',  'PDP Flag', 'URL_Cat']]
sal_data.columns = ['url','index','UUID',  'PDP Flag', 'URL_Cat']

In [25]:
sal_data 

,url,index,UUID,PDP Flag,URL_Cat
0,https://www.target.com/,0,LiveIHbr8thAprGDCPITAW1975,0.0,Home Page
1,https://www.target.com/s?searchTerm=hair+removal,1,LiveIHbr8thAprGDCPITAW1975,0.0,Search Page
2,https://www.target.com/p/spa-sciences-sima-son...,2,LiveIHbr8thAprGDCPITAW1975,1.0,PDP
3,https://www.amazon.com/s?crid=1TL9445LWNRYU&k=...,3,LiveIHbr8thAprCYFRNNAV843,0.0,Search Page
4,https://www.amazon.com/Philips-Norelco-Shaver-...,4,LiveIHbr8thAprCYFRNNAV843,1.0,PDP
...,...,...,...,...,...
21737,NaN,21737,NaN,NaN,NaN
21738,NaN,21738,NaN,NaN,NaN
21739,NaN,21739,NaN,NaN,NaN
21740,NaN,21740,NaN,NaN,NaN


In [26]:
# take data from row 3917
# sal_data = sal_data.iloc[3917:]

In [27]:
sal_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21742 entries, 0 to 21741
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   url       20004 non-null  object 
 1   index     21742 non-null  int64  
 2   UUID      19920 non-null  object 
 3   PDP Flag  20003 non-null  float64
 4   URL_Cat   10872 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 849.4+ KB


In [28]:
sal_data

,url,index,UUID,PDP Flag,URL_Cat
0,https://www.target.com/,0,LiveIHbr8thAprGDCPITAW1975,0.0,Home Page
1,https://www.target.com/s?searchTerm=hair+removal,1,LiveIHbr8thAprGDCPITAW1975,0.0,Search Page
2,https://www.target.com/p/spa-sciences-sima-son...,2,LiveIHbr8thAprGDCPITAW1975,1.0,PDP
3,https://www.amazon.com/s?crid=1TL9445LWNRYU&k=...,3,LiveIHbr8thAprCYFRNNAV843,0.0,Search Page
4,https://www.amazon.com/Philips-Norelco-Shaver-...,4,LiveIHbr8thAprCYFRNNAV843,1.0,PDP
...,...,...,...,...,...
21737,NaN,21737,NaN,NaN,NaN
21738,NaN,21738,NaN,NaN,NaN
21739,NaN,21739,NaN,NaN,NaN
21740,NaN,21740,NaN,NaN,NaN


In [1]:
def get_pdp(URL):
try:
    if URL.count("google",0,35) or URL.count("yahoo",0,35) or URL.count("bing",0,35) or URL.count("youtube",0,35) or URL.count("facebook",0,35) or URL.count("wikipedia",0,35)> 0:
        class_pdp = "search_page"
    elif len(URL) < 35:
        class_pdp = "skipped"
    elif all(x in URL for x in ['amazon','/gp/aw/d/']) or \
        all(x in URL for x in ['amazon','/dp/']) or \
        all(x in URL for x in ['target','/p/']) or \
        all(x in URL for x in ['walmart','/ip/']) or \
        all(x in URL for x in ['walmart','track']):
        class_pdp = "YES"
    elif all(x in URL for x in ['amazon','/stores/']) or \
        all(x in URL for x in ['target','/b/']) or \
        all(x in URL for x in ['amazon','i=']) or \
        all(x in URL for x in ['target','/c/']) or \
        all(x in URL for x in ['walmart','browse']) or \
        all(x in URL for x in ['walmart','/cp']) or \
        all(x in URL for x in ['target','searchTerm']) or \
        all(x in URL for x in ['walmart','search']) or \
        all(x in URL for x in ['amazon','/goldbox']) or \
        all(x in URL for x in ['amazon','/offers']) or \
        all(x in URL for x in ['amazon','/angebot']) or \
        all(x in URL for x in ['target','/top-deals']) or \
        all(x in URL for x in ['target','/circle']) or \
        all(x in URL for x in ['walmart','/m']) or \
        all(x in URL for x in ['walmart','/deals']) or \
        all(x in URL for x in ['amazon','Best-Sellers']) or \
        all(x in URL for x in ['amazon','/ask','/questions']) or \
        all(x in URL for x in ['amazon','/product-reviews/']) or \
        all(x in URL for x in ['amazon','/c/ref=mw_dp_buy_crt']) or \
        all(x in URL for x in ['amazon','/cart/']) or \
        all(x in URL for x in ['amazon','/huc/view.html?']) or \
        all(x in URL for x in ['target','/co-cart']) or \
        all(x in URL for x in ['target','/cart']) or \
        all(x in URL for x in ['amazon','bestsellers','Best-Sellers']):
        class_pdp = "Other_pages"
    else:
        HEADERS = ({'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36','Accept-Language': 'en-US, en;q=0.5'})
        webpage = requests.get(URL, headers=HEADERS, timeout=25)
        soup = BeautifulSoup(webpage.content, "html5lib")
        text = soup.find_all(text=True)
        if text == None:
            class_pdp = "No Data"
        else:
            new = []
            for i in text:
                new.append(i.lower())
                mystr =" ".join(new)
            pdp = [i for i in mystr if i == "add to cart" or i == " add to cart " or i == "add to shipping" or 
            i == " add to shipping " or i == "add to basket" or i == " add to basket " or i == "buy now" or 
            i == " buy now " or i == "add item" or i == " add item " or i == "in the cart" or i == " in the cart " or 
            i == "description" or i == "item description" or i == "product description" or i == "about this product" or 
            i == "about this item" or i == "overview"  or i == "over view" or i == " add for shipping " or i == "add for shipping"]
            if len(pdp) > 0 and len(pdp)< 10:
                class_pdp = "YES"
            elif mystr.count("add to basket" or "add to shipping" or "add to cart" or "buy now" or "add item" or "in the cart" or " add to cart " or " add to shipping " or " add to basket " or " buy now " or " add item " or " in the cart " or "add for shipping " or " add for shipping ")> 0 \
                & mystr.count("add to basket" or "add to shipping" or "add to cart" or "buy now" or "add item" or "in the cart" or " add to cart " or " add to shipping " or " add to basket " or " buy now " or " add item " or " in the cart " or "add for shipping " or " add for shipping ")< 15:
                class_pdp = "YES"
            else:
                class_pdp = "NO"
except Exception as e:
    class_pdp = "error or timeout"
return class_pdp




IndentationError: expected an indented block (2086650066.py, line 2)

In [ ]:
def get_pdp(URL):
    try :
        

In [ ]:
for i in sal_data.url:
    class_pdp = get_pdp(i)
    f = open('class_pdp2.txt', 'a+')
    f.write(class_pdp+"\n")
    f.close()

In [31]:
f2 = open("class_pdp2.txt", "r")
pdp_tag = f2.readlines()

pdp=[]
for i in pdp_tag:
    pdp.append(i[:-1])

In [32]:
pdp_df = pd.DataFrame(pdp, columns=['pdp'])
# add and index column starting from 10871
pdp_df['index'] = sal_data['index']
len_pfp = len(pdp_df)

In [33]:
len_pfp

4345

In [34]:
pdp_df

,pdp,index
0,skipped,0
1,Other_pages,1
2,YES,2
3,NO,3
4,YES,4
...,...,...
4340,YES,4340
4341,NO,4341
4342,search_page,4342
4343,search_page,4343


In [35]:
merged_df = pd.merge(sal_data, pdp_df, on='index')

In [36]:
merged_df

,url,index,UUID,PDP Flag,URL_Cat,pdp
0,https://www.target.com/,0,LiveIHbr8thAprGDCPITAW1975,0.0,Home Page,skipped
1,https://www.target.com/s?searchTerm=hair+removal,1,LiveIHbr8thAprGDCPITAW1975,0.0,Search Page,Other_pages
2,https://www.target.com/p/spa-sciences-sima-son...,2,LiveIHbr8thAprGDCPITAW1975,1.0,PDP,YES
3,https://www.amazon.com/s?crid=1TL9445LWNRYU&k=...,3,LiveIHbr8thAprCYFRNNAV843,0.0,Search Page,NO
4,https://www.amazon.com/Philips-Norelco-Shaver-...,4,LiveIHbr8thAprCYFRNNAV843,1.0,PDP,YES
...,...,...,...,...,...,...
4340,https://www.amazon.com/dp/B003H3G470?ascsubtag...,4340,F432312A241F6DCA2AECF280674B339AD,1.0,PDP,YES
4341,https://www.byrdie.com/best-epilators-4767513,4341,F432312A241F6DCA2AECF280674B339AD,0.0,NaN,NO
4342,https://www.google.com/search?q=best+shampoo+f...,4342,F432312A241F6DCA2AECF280674B339AD,0.0,NaN,search_page
4343,https://www.google.com/search?q=best+shampoo+f...,4343,F432312A241F6DCA2AECF280674B339AD,0.0,NaN,search_page


In [37]:
df = pd.read_csv('ClickStream+PurchasesCombined_FINAL_15Feb (1).csv')
# Creat a new column for index
df['index'] = df.index

c:\VS_Code\O360\O360\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (21,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
df.columns

Index(['Name', 'Total time spent on user test', 'Average time taken on tasks',
       'Average page views per task', 'User agent', 'Custom ID',
       'Date completed user test', 'Item', 'Task/Question', 'Page views',
       'Path Analysis', 'Navigation', 'Time spent', 'Task result', 'domain',
       'UUID', 'surveyID', 'TouchPoint', 'Sequence', 'Reverse Sequence',
       'Sponsored', 'Sponsored link', 'Domain_Type', 'DTC_Page',
       'Domain_Type2', 'SearchType', 'SearchType2', 'Gender', 'Country',
       'Survey', 'FullName', 'JourneyStage', 'ProdName', 'ratings', 'OID',
       'resid', 'User Id', 'searchTerm', 'PDP Flag', 'Prev_PDP', 'Prev_Brand',
       'Brand', 'URL_Cat', 'index'],
      dtype='object')

In [39]:
df = df[['Path Analysis','index','UUID',  'PDP Flag', 'URL_Cat']]

In [40]:
df_ckeck = pd.merge(df, merged_df, on='index')

In [41]:
df_ckeck.to_csv('df_ckeck2.csv')